In [ ]:
from osgeo import ogr, osr
import networkx as nx
from matplotlib import pyplot as plt
%matplotlib inline

driver = ogr.GetDriverByName('ESRI Shapefile')
filepath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RI_converted.shp'
dataset = driver.Open(filepath)

# from Layer
layer = dataset.GetLayer()
spatialRef = layer.GetSpatialRef()
# from Geometry
feature = layer.GetNextFeature()
geom = feature.GetGeometryRef()
spatialRef = geom.GetSpatialReference()

shp = ogr.Open(filepath)

In [ ]:
g = read_shp2(filepath, ['RoadSegmen', 'AssignedLe'], simplify=True)

In [ ]:
first = g.edges()[0]
g.edge[(-71.10891187098966, 42.474431813774984)][(-71.10750084168293, 42.47446692343721)]['AssignedLe']

In [ ]:
g2 = g
nx.convert_node_labels_to_integers(g, label_attribute="oldLabels")

In [ ]:
g.node

In [ ]:
it = 0
for n in g.nodes_iter():
    it+=1
    if (it%100 == 0):
        print(n)

In [ ]:
def read_shp2(path, fields, simplify=True):
    """Generates a networkx.Graph from shapefiles. Point geometries are
    translated into nodes, lines into edges. Coordinate tuples are used as
    keys. Attributes are preserved, line geometries are simplified into start
    and end coordinates. Accepts a single shapefile or directory of many
    shapefiles.

    "The Esri Shapefile or simply a shapefile is a popular geospatial vector
    data format for geographic information systems software [1]_."

    Parameters
    ----------
    path : file or string
       File, directory, or filename to read.
    
    fields: list
       A list of fields to pass on to the graph edges. 

    simplify:  bool
        If ``True``, simplify line geometries to start and end coordinates.
        If ``False``, and line feature geometry has multiple segments, the 
        non-geometric attributes for that feature will be repeated for each 
        edge comprising that feature.

    Returns
    -------
    G : NetworkX graph

    Examples
    --------
    >>> G=nx.read_shp('test.shp') # doctest: +SKIP

    References
    ----------
    .. [1] http://en.wikipedia.org/wiki/Shapefile
    """
    try:
        from osgeo import ogr
    except ImportError:
        raise ImportError("read_shp requires OGR: http://www.gdal.org/")

    if not isinstance(path, str):
        return

    net = nx.Graph()
    shp = ogr.Open(path)
    for lyr in shp:
        # fields = [x.GetName() for x in lyr.schema]
        layerit = 0
        nbad = 0
        ngood = 0
        for f in lyr:
            layerit += 1
            flddata = [f.GetField(f.GetFieldIndex(x)) for x in fields]
            g = f.geometry()
            if g is None:
                print('g is None, layer iteration: {0}'.format(str(layerit)))
                nbad += 1
                continue
            attributes = dict(zip(fields, flddata))
            attributes["ShpName"] = lyr.GetName()
            if g.GetGeometryType() == 1:  # point
                net.add_node((g.GetPoint_2D(0)), attributes)
            if g.GetGeometryType() == 2:  # linestring
                last = g.GetPointCount() - 1
                if simplify:
                    attributes["Wkb"] = g.ExportToWkb()
                    attributes["Wkt"] = g.ExportToWkt()
                    attributes["Json"] = g.ExportToJson()
                    net.add_edge(g.GetPoint_2D(0), g.GetPoint_2D(last), attributes)
                else:
                    # separate out each segment as individual edge
                    for i in range(last):
                        pt1 = g.GetPoint_2D(i)
                        pt2 = g.GetPoint_2D(i + 1)
                        segment = ogr.Geometry(ogr.wkbLineString)
                        segment.AddPoint_2D(pt1[0], pt1[1])
                        segment.AddPoint_2D(pt2[0], pt2[1])
                        attributes["Wkb"] = segment.ExportToWkb()
                        attributes["Wkt"] = segment.ExportToWkt()
                        attributes["Json"] = segment.ExportToJson()
                        del segment
                        net.add_edge(pt1, pt2, attributes)
            ngood += 1
        print('good = {0}, bad = {1}'.format(str(ngood), str(nbad)))
    return net

In [ ]:
g.to_undirected()

In [ ]:
n = g.nodes()

p = nx.shortest_path(g, source=n[0], target = n[3])
len(p)

In [ ]:
import networkx as nx
G = nx.complete_graph(5)
from networkx.drawing.nx_agraph import graphviz_layout
pos = graphviz_layout(G)
nx.draw(G, pos)

In [ ]:
help(nx.Graph.add_edge)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon

In [ ]:
import graphHandler as gh

graph = gh.NetworkGenerator()
filepath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RI_converted.shp'
graph.loadGraph(filepath=filepath, fields=['RoadSegmen', 'AssignedLe'], simplify=False)

g is None, layer iteration: 7508
g is None, layer iteration: 8378
g is None, layer iteration: 8843
g is None, layer iteration: 10532
g is None, layer iteration: 33857
g is None, layer iteration: 36465
g is None, layer iteration: 95844
g is None, layer iteration: 101579
g is None, layer iteration: 101660
g is None, layer iteration: 102926
g is None, layer iteration: 106601
g is None, layer iteration: 130559
g is None, layer iteration: 131374
g is None, layer iteration: 131414


In [6]:
graph.net